In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numerate import *
from numerate.limiters import *
from numerate.functions import *
from numerate.verification import *

In [ ]:
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams["animation.embed_limit"] = 30.0

In [ ]:
g = lambda x : tophat(x, b=0.2, c=0.7)
h = lambda x : gaussian(x, b=0.05, c=0.3)
f = lambda x : g(x) + h(x)

In [ ]:
a = 1
x0 = 0
x1 = 1
xs = 300
revolutions = 3
ts = 500 * revolutions
func = f

In [ ]:
def solve(solver, *, filename=None, **kwargs):
    eq = solver(
    a,
    func,
    x0=x0,
    x1=x1,
    xs=xs,
    revolutions=revolutions,
    ts=ts,
    **kwargs,
    )
    solution = eq.solve()
    
    return eq.animate(solution, drop=3, interval=20, filename=filename)

In [ ]:
solve(
    NumericalAdvectionEquationUpwindForward,
    filename='figures/presentation/upwind-forward-anim.gif',
)

In [ ]:
solve(
    NumericalAdvectionEquationFluxLimiter, 
    filename='figures/presentation/lax-wendroff-anim.gif', 
    phi=lax_wendroff,
)

In [ ]:
solve(
    NumericalAdvectionEquationFluxLimiter, 
    filename='figures/presentation/flux-limiter-fromm-anim.gif', 
    phi=fromm,
)

In [ ]:
solve(
    NumericalAdvectionEquationFluxLimiter, 
    filename='figures/presentation/flux-limiter-van-leer-anim.gif', 
    phi=van_leer,
)

In [ ]:
def plot_tvd(filename, *args):
    fig, ax = plt.subplots()
    
    r = np.linspace(-1, 5, 3000)
    
    func1 = np.piecewise(
        r,
        [np.logical_and(r > 0, r < 0.5), r >= 0.5],
        [lambda x: 2 * x, lax_wendroff]
    )
    
    func2 = np.piecewise(
        r,
        [np.logical_and(r > 0, r < 2), r >= 2],
        [beam_warming, lambda x: 2]
    )
    
    ax.fill_between(
        r, 
        func1,
        func2,
        where=r > 0,
        color='black',
        alpha=0.1,
    )
    
    for phi in args:
        ax.plot(r, phi(r), label=phi.__name__.replace('_', ' ').title())
        
    if len(args) > 1:
        ax.legend(loc='lower right', fontsize=12)
    
    ax.set_xlabel(r'$r$')
    ax.set_ylabel(r'$\phi\left(r\right)$')
    ax.set_xlim([-1, 5])
    ax.set_ylim([-1, 3])
    
    ax.spines['left'].set_position('zero')
    ax.spines['right'].set_color('none')

    ax.spines['bottom'].set_position('zero')
    ax.spines['top'].set_color('none')
    
    plt.savefig(filename)
    plt.show()

In [ ]:
plot_tvd('figures/presentation/tvd-region.pdf', lax_wendroff, fromm, van_leer)

In [ ]:
xs = np.linspace(0, 1, 1000)
for i in range(-3, 2):
    ys = xs + i * 0.5
    if i > 0:
        plt.plot(xs, ys, label=f"$y = a x + {i * 0.5}$")
    elif i < 0:
        plt.plot(xs, ys, label=f"$y = a x {i * 0.5}$")
    else:
        plt.plot(xs, ys, label=f"$y = a x$")
    plt.xlim(0, 1)
    plt.ylim(-1, 1)
    plt.legend(loc="lower right")

plt.savefig("figures/presentation/characteristics.pdf")